# Парсинг текстов с сайтов и соцсетей

Этот блокнот загружает скрипты из репозитория и помогает быстро получить тексты из:
- статей Фонтанки
- отзывов Яндекса
- групп ВК

Ниже — установка зависимостей, импорт функций и интерактивные формы для ввода URL/токенов.


In [ ]:
#@title Установка зависимостей
!pip -q install requests beautifulsoup4 lxml ipywidgets pandas

import os
import sys
from pathlib import Path

# Укажите репозиторий в формате "owner/repo" при необходимости
REPO_SLUG = os.environ.get("GITHUB_REPO", "Sandrro/digital_identity")
REPO_URL = f"https://github.com/{REPO_SLUG}"
REPO_DIR = Path("/content/digital_identity")

if not REPO_DIR.exists():
    !git clone --depth 1 {REPO_URL} {REPO_DIR}

sys.path.insert(0, str(REPO_DIR))


In [ ]:
#@title Импорт парсеров
import sys
sys.path.insert(0, "/content/digital_identity/parsers")
from fontanka_parser import fetch_fontanka_article
from vk_group_parser import VKGroupParser
from yandex_reviews_parser import fetch_yandex_reviews


In [ ]:
#@title Интерактивный запуск
import csv
from datetime import datetime, timezone
from pathlib import Path

import ipywidgets as widgets
import pandas as pd
from IPython.display import display

OUTPUT_CSV = Path("/content/digital_identity/parsed_texts.csv")

parser_dropdown = widgets.Dropdown(
    options=[
        ("Фонтанка", "fontanka"),
        ("Яндекс отзывы", "yandex"),
        ("ВК группа", "vk"),
    ],
    description="Парсер:"
)
url_input = widgets.Text(
    value="",
    description="URL/домен:",
    placeholder="https://...",
    layout=widgets.Layout(width="80%"),
)
token_input = widgets.Password(
    value="",
    description="VK токен:",
    placeholder="Требуется только для ВК",
)
max_items = widgets.IntSlider(
    value=20,
    min=1,
    max=200,
    step=1,
    description="Лимит:"
)
run_button = widgets.Button(description="Запустить")
output = widgets.Output()

def _format_vk_timestamp(ts):
    if ts is None:
        return None
    return datetime.fromtimestamp(ts, tz=timezone.utc).isoformat()

def _write_rows(rows):
    if not rows:
        return
    OUTPUT_CSV.parent.mkdir(parents=True, exist_ok=True)
    with OUTPUT_CSV.open("w", newline="", encoding="utf-8") as handle:
        writer = csv.DictWriter(
            handle,
            fieldnames=["source", "url", "title", "timestamp", "text"],
        )
        writer.writeheader()
        writer.writerows(rows)
    print(f"CSV сохранен: {OUTPUT_CSV}")

def run_parser(_):
    output.clear_output()
    with output:
        rows = []
        parser = parser_dropdown.value
        if parser == "fontanka":
            article = fetch_fontanka_article(url_input.value)
            rows.append({
                "source": "fontanka",
                "url": article.url,
                "title": article.title,
                "timestamp": article.published_at,
                "text": article.text,
            })
            if article.title:
                print(article.title)
            if article.published_at:
                print(article.published_at)
            print(article.text[:3000])
        elif parser == "yandex":
            reviews = fetch_yandex_reviews(url_input.value, max_reviews=max_items.value)
            for review in reviews:
                rows.append({
                    "source": "yandex",
                    "url": url_input.value,
                    "title": None,
                    "timestamp": None,
                    "text": review.text,
                })
                print(review.text)
                print("-" * 20)
        else:
            if not token_input.value:
                raise ValueError("Нужен VK токен.")
            vk = VKGroupParser(token_input.value)
            for post in vk.iter_posts(url_input.value, total=max_items.value):
                timestamp = _format_vk_timestamp(post.date)
                rows.append({
                    "source": "vk",
                    "url": url_input.value,
                    "title": None,
                    "timestamp": timestamp,
                    "text": post.text,
                })
                if timestamp:
                    print(timestamp)
                print(post.text)
                print("-" * 20)
        _write_rows(rows)
        if rows:
            display(pd.DataFrame(rows))

run_button.on_click(run_parser)
display(parser_dropdown, url_input, token_input, max_items, run_button, output)
